In [ ]:
# !pip install tf2onnx

In [12]:
# Tensorflow to Pytorch
import tensorflow as tf
import importlib
import sys
sys.path.append("../models")

task_name = 'fashion_mnist'
task_type = 'mnist'
model_size = 'medium'

In [13]:
model = getattr(
    importlib.import_module(task_type),
    model_size
    )()
model.load_weights(os.path.join('.', f'{task_name}', f'{model_size}.h5'))
tf.saved_model.save(model, os.path.join('.', f'{task_name}', f'{model_size}'))

INFO:tensorflow:Assets written to: ./fashion_mnist/medium/assets


INFO:tensorflow:Assets written to: ./fashion_mnist/medium/assets


In [14]:
saved_model = os.path.join('.', task_name, model_size)
out_model = os.path.join('.', task_name, model_size+'.onnx')
!python -m tf2onnx.convert --saved-model $saved_model --output $out_model

2022-07-13 04:44:00.146950: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-07-13 04:44:01.556120: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-13 04:44:02.086547: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:

In [1]:
import tensorflow as tf
import numpy as np
import importlib
import sys
sys.path.append("..")

In [4]:
task_name = 'mnist'
model_size = 'medium'

In [5]:
def dataset_screening(
    victim_model: tf.keras.Model,
    dataset: tf.data.Dataset
) -> tf.data.Dataset:

    correct_x = []
    correct_t = []
    for batch in dataset:
        x, t = batch
        t = t.numpy()
        y = np.argmax(victim_model(x, training=False).numpy(), axis=1)
        idx = (t == y)
        correct_x.append(x.numpy()[idx])
        correct_t.append(t[idx])
    correct_x = np.concatenate(correct_x, axis=0)
    correct_t = np.concatenate(correct_t, axis=0)

    return tf.data.Dataset.from_tensor_slices((correct_x, correct_t))

In [6]:
cfg = {
    'task_name': task_name,
    'ds_path':'/dataset/'+task_name,
    'batch_size':256
    }

In [9]:
_, ds_test = getattr(
    importlib.import_module('datasets.'+task_type),
    'get_dataset'
    )(cfg)

In [10]:
# Load Victim model
victim_model = getattr(
    importlib.import_module('models.'+task_type),
    model_size
    )()
victim_model.load_weights(os.path.join(
        '.',
        task_name,
        model_size+'.h5'
    ))

In [12]:
# Datasetのスクリーニング
ds_test = dataset_screening(
    victim_model,
    ds_test
).take(5000)
x = np.concatenate([[v[0].numpy()] for v in ds_test], axis=0)
t = np.concatenate([[v[1]] for v in ds_test])
np.savez(os.path.join('.', task_name, 'ds_test.npz'), x=x, t=t)